In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
from pathlib import Path

# required to enable non-blocking interaction:
%gui qt5

from copy import deepcopy
from numba import jit
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from attrs import define, field, fields, Factory
import tables as tb
from datetime import datetime, timedelta

# Pho's Formatting Preferences
from pyphocorehelpers.preferences_helpers import set_pho_preferences, set_pho_preferences_concise, set_pho_preferences_verbose
set_pho_preferences_concise()

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager

# NeuroPy (Diba Lab Python Repo) Loading
# from neuropy import core
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.core.epoch import NamedTimerange, Epoch
from neuropy.core.ratemap import Ratemap
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_file_only, matplotlib_configuration, matplotlib_configuration_update
from neuropy.core.neuron_identities import NeuronIdentityTable, neuronTypesList, neuronTypesEnum

## For computation parameters:
from neuropy.analyses.placefields import PlacefieldComputationParameters
from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core.neurons import NeuronType
from neuropy.core.user_annotations import UserAnnotationsManager
from neuropy.core.position import Position
from neuropy.core.session.dataSession import DataSession
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot

from pyphocorehelpers.print_helpers import print_object_memory_usage, print_dataframe_memory_usage, print_value_overview_only, DocumentationFilePrinter, print_keys_if_possible, generate_html_string, CapturedException

# pyPhoPlaceCellAnalysis:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import NeuropyPipeline # get_neuron_identities
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import export_pyqtgraph_plot
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_load_session, batch_extended_computations, batch_extended_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap
import pyphoplacecellanalysis.External.pyqtgraph as pg

from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_perform_all_plots
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import _find_any_context_neurons
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler # for `post_compute_validate(...)`
from neuropy.utils.mixins.AttrsClassHelpers import AttrsBasedClassHelperMixin, serialized_field, serialized_attribute_field, non_serialized_field
from neuropy.utils.mixins.HDF5_representable import HDF_DeserializationMixin, post_deserialize, HDF_SerializationMixin, HDFMixin
from pyphoplacecellanalysis.Analysis.Decoder.reconstruction import BasePositionDecoder
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends # for `_perform_long_short_instantaneous_spike_rate_groups_analysis`
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import SingleBarResult, InstantaneousSpikeRateGroupsComputation # for `BatchSessionCompletionHandler`, `AcrossSessionsAggregator`

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# import pylustrator # call `pylustrator.start()` before creating your first figure in code.
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Mixins.DataSeriesColorHelpers import UnitColoringMode, DataSeriesColorHelpers
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import _build_default_tick, build_scatter_plot_kwargs
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.Render2DScrollWindowPlot import Render2DScrollWindowPlotMixin, ScatterItemData
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_programmatic_figures, batch_programmatic_figures
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_add_replay_epoch_rasters, PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_3, main_complete_figure_generations

global_data_root_parent_path = find_first_extant_path([Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data')])
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


# Load Pipeline

In [2]:
%pdb off
# ==================================================================================================================== #
# Load Data                                                                                                            #
# ==================================================================================================================== #

active_data_mode_name = 'kdiba'
local_session_root_parent_context = IdentifyingContext(format_name=active_data_mode_name) # , animal_name='', configuration_name='one', session_name=a_sess.session_name
local_session_root_parent_path = global_data_root_parent_path.joinpath('KDIBA')

# [*] - indicates bad or session with a problem
# 0, 1, 2, 3, 4, 5, 6, 7, [8], [9], 10, 11, [12], 13, 14, [15], [16], 17, 
# curr_context: IdentifyingContext = good_contexts_list[1] # select the session from all of the good sessions here.
curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15') # DONE. Very good. Many good Pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43') # DONE, might be the BEST SESSION, good example session with lots of place cells, clean replays, and clear bar graphs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31') # DONE, Good Pfs but no good replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6') # BAD, 2023-07-14, unsure why still.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19') # DONE, GREAT, both good Pfs and replays!
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25') # DONE, Added replay selections. Very "jumpy" between the starts and ends of the track.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40') # DONE, Added replay selections. A TON of putative replays in general, most bad, but some good.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46') # DONE, added replay selections. Very few (like 12) replays each.
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-13_15-22-3') # DONE, Good Pfs, no good epochs.
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30') # DONE, okay replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50') # DONE, very few replays (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-19_13-34-40') # BAD
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54') # DONE, one replay each (selected)
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3') # BAD, Good Pfs strangely despite horrible map, no good epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-12_15-25-59') # BAD, No Epochs
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-16_18-47-52')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-17_12-52-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-25_13-20-55')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-28_12-38-13')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44') # DONE, good. Many good pfs, many good replays.
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0') # DONE, good?, replays selected, few
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25') # DONE, very few replays
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_12-15-3')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-09_22-4-5')
# curr_context = IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54') # DONE, replays selected, quite a few replays but few are very good.

local_session_parent_path: Path = local_session_root_parent_path.joinpath(curr_context.animal, curr_context.exper_name) # 'gor01', 'one' - probably not needed anymore
basedir: Path = local_session_parent_path.joinpath(curr_context.session_name).resolve()
print(f'basedir: {str(basedir)}')

# Read if possible:
saving_mode = PipelineSavingScheme.SKIP_SAVING
force_reload = False

# # Force write:
# # saving_mode = PipelineSavingScheme.TEMP_THEN_OVERWRITE
# saving_mode = PipelineSavingScheme.OVERWRITE_IN_PLACE
# force_reload = True

## TODO: if loading is not possible, we need to change the `saving_mode` so that the new results are properly saved.

# ==================================================================================================================== #
# Load Pipeline                                                                                                        #
# ==================================================================================================================== #
# with VizTracer(output_file=f"viztracer_{get_now_time_str()}-full_session_LOO_decoding_analysis.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
# epoch_name_includelist = ['maze']
epoch_name_includelist = None
active_computation_functions_name_includelist=[#'_perform_estimated_epochs_computation',  # AL:WAYS OFF
                                            '_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation', # AL:WAYS OFF
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation' # AL:WAYS OFF
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding' # AL:WAYS OFF
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']

curr_active_pipeline: NeuropyPipeline = batch_load_session(global_data_root_parent_path, active_data_mode_name, basedir, epoch_name_includelist=epoch_name_includelist,
                                        computation_functions_name_includelist=active_computation_functions_name_includelist,
                                        saving_mode=saving_mode, force_reload=force_reload,
                                        skip_extended_batch_computations=True, debug_print=False, fail_on_exception=False) # , active_pickle_filename = 'loadedSessPickle_withParameters.pkl'


if not force_reload: # not just force_reload, needs to recompute whenever the computation fails.
    try:
        curr_active_pipeline.load_pickled_global_computation_results()
    except Exception as e:
        exception_info = sys.exc_info()
        e = CapturedException(e, exception_info)
        print(f'cannot load global results: {e}')
        raise

# 6m 1.1s
# 12m 15.6s

try:
    ## Post Compute Validate 2023-05-16:
    was_updated = BatchSessionCompletionHandler.post_compute_validate(curr_active_pipeline) ## TODO: need to potentially re-save if was_updated
    if was_updated:
        try:
            curr_active_pipeline.save_pipeline(saving_mode=saving_mode)
        except Exception as e:
            ## TODO: catch/log saving error and indicate that it isn't saved.
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'ERROR RE-SAVING PIPELINE after update. error: {e}')

    curr_active_pipeline.reload_default_computation_functions()
    extended_computations_include_includelist=['long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding', 
    #  'long_short_rate_remapping',
     'long_short_inst_spike_rate_groups',
    ] # do only specifiedl , 'long_short_rate_remapping'
    force_recompute_global = force_reload
    # force_recompute_global = True
    newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=force_recompute_global, debug_print=False)
    if (len(newly_computed_values) > 0) and (saving_mode.value != 'skip_saving'):
        print(f'newly_computed_values: {newly_computed_values}. Saving global results...')
        try:
            # curr_active_pipeline.global_computation_results.persist_time = datetime.now()
            # Try to write out the global computation function results:
            curr_active_pipeline.save_global_computation_results()
        except Exception as e:
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'\n\n!!WARNING!!: saving the global results threw the exception: {e}')
            print(f'\tthe global results are currently unsaved! proceed with caution and save as soon as you can!\n\n\n')
    else:
        print(f'no changes in global results.')

except Exception as e:
    exception_info = sys.exc_info()
    e = CapturedException(e, exception_info)
    print(f'second half threw: {e}')

Automatic pdb calling has been turned OFF
basedir: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15
Loading loaded session pickle file results : W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl... done.
Loading pickled pipeline success: W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['_perform_baseline_placefield_computation', '_perform_extended_statistics_computation', '_perform_position_decoding_computation', '_perform_firing_rate_trends_computation', '_perform_pf_find_ratemap_peaks_computation', '_perform_two_step_position_decoding_computation']
ignored_changed_filters_list: ['maze1', 'maze2', 'maze']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist

In [ ]:
curr_active_pipeline.save_global_computation_results()

# End Run

In [5]:
## long_short_decoding_analyses:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
## Extract variables from results object:
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global

# (long_one_step_decoder_1D, short_one_step_decoder_1D), (long_one_step_decoder_2D, short_one_step_decoder_2D) = compute_short_long_constrained_decoders(curr_active_pipeline, recalculate_anyway=True)
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_epoch_context, short_epoch_context, global_epoch_context = [curr_active_pipeline.filtered_contexts[a_name] for a_name in (long_epoch_name, short_epoch_name, global_epoch_name)]
long_epoch_obj, short_epoch_obj = [Epoch(curr_active_pipeline.sess.epochs.to_dataframe().epochs.label_slice(an_epoch_name)) for an_epoch_name in [long_epoch_name, short_epoch_name]]
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_computation_config, short_computation_config, global_computation_config = [curr_active_pipeline.computation_results[an_epoch_name]['computation_config'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
long_pf1D, short_pf1D, global_pf1D = long_results.pf1D, short_results.pf1D, global_results.pf1D
long_pf2D, short_pf2D, global_pf2D = long_results.pf2D, short_results.pf2D, global_results.pf2D
decoding_time_bin_size = long_one_step_decoder_1D.time_bin_size # 1.0/30.0 # 0.03333333333333333
long_pf1D_dt, short_pf1D_dt, global_pf1D_dt = long_results.pf1D_dt, short_results.pf1D_dt, global_results.pf1D_dt
long_pf2D_dt, short_pf2D_dt, global_pf2D_dt = long_results.pf2D_dt, short_results.pf2D_dt, global_results.pf2D_dt
# global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
# global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt


## Get global 'long_short_post_decoding' results:
curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
expected_v_observed_result, curr_long_short_rr = curr_long_short_post_decoding.expected_v_observed_result, curr_long_short_post_decoding.rate_remapping
rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df
Flat_epoch_time_bins_mean, Flat_decoder_time_bin_centers, num_neurons, num_timebins_in_epoch, num_total_flat_timebins, is_short_track_epoch, is_long_track_epoch, short_short_diff, long_long_diff = expected_v_observed_result.Flat_epoch_time_bins_mean, expected_v_observed_result.Flat_decoder_time_bin_centers, expected_v_observed_result.num_neurons, expected_v_observed_result.num_timebins_in_epoch, expected_v_observed_result.num_total_flat_timebins, expected_v_observed_result.is_short_track_epoch, expected_v_observed_result.is_long_track_epoch, expected_v_observed_result.short_short_diff, expected_v_observed_result.long_long_diff

jonathan_firing_rate_analysis_result: JonathanFiringRateAnalysisResult = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

inst_spike_rate_groups_result: InstantaneousSpikeRateGroupsComputation = curr_active_pipeline.global_computation_results.computed_data.long_short_inst_spike_rate_groups

DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computed_data
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config
DEPRICATION WARNING: workaround to allow subscripting ComputationResult objects. Will be depricated. key: computation_config


In [6]:
# Find instantaneous firing rate for spikes outside of replays
# 2023-09-06 - Method Kamran and I dicusssed in his office last Thursday
# Uses instantaneous firing rates for each cell computed during any non-replay epoch. Kamran was concerned that some cells fire spikes on the end platforms during either short or long, but because we're only considering spikes that contribute to placefeields (of which the endcap spikes are omitted due to velocity requirements) these cells are said to be long/short exclusive despite firing frequently on the end caps.


from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import _fr_index

# Sort by column: 'custom_frs_index' (ascending)
custom_InstSpikeRateTrends_df = inst_spike_rate_groups_result.all_incl_endPlatforms_InstSpikeRateTrends_df
custom_InstSpikeRateTrends_df

# Calculate 10th and 90th percentiles
lower_bound = custom_InstSpikeRateTrends_df['custom_frs_index'].quantile(0.10)
upper_bound = custom_InstSpikeRateTrends_df['custom_frs_index'].quantile(0.90)

# Filter rows
bottom_10_percent = custom_InstSpikeRateTrends_df[custom_InstSpikeRateTrends_df['custom_frs_index'] <= lower_bound]
top_10_percent = custom_InstSpikeRateTrends_df[custom_InstSpikeRateTrends_df['custom_frs_index'] >= upper_bound]



In [7]:
top_10_percent

,aclu,long_inst_fr,short_inst_fr,custom_frs_index
33,35,37.532016,25.381719,0.193126
6,8,14.073102,8.932192,0.223466
14,16,22.895041,14.317355,0.230506
71,73,11.975029,6.946617,0.265749
57,59,37.315187,21.601736,0.266705
26,28,14.273504,7.619963,0.303905
80,82,18.627768,9.847235,0.308359
107,109,8.709319,4.293549,0.339600
103,105,2.062792,0.536694,0.587077
95,97,18.924973,2.542687,0.763115


In [8]:
bottom_10_percent

,aclu,long_inst_fr,short_inst_fr,custom_frs_index
2,4,0.571960,22.098043,-0.949540
11,13,1.659619,10.391941,-0.724580
17,19,5.361046,23.194960,-0.624524
1,3,5.674267,22.709402,-0.600174
22,24,4.345638,13.252129,-0.506115
54,56,13.255795,36.612280,-0.468365
56,58,8.993553,24.386682,-0.461145
13,15,4.278449,11.089273,-0.443190
23,25,9.533276,23.887817,-0.429505
83,85,11.820699,21.429137,-0.288977


In [ ]:
y_frs_index = {aclu:_fr_index(long_mean_laps_frs[aclu], short_mean_laps_frs[aclu]) for aclu in long_mean_laps_frs.keys()}
x_frs_index = {aclu:_fr_index(long_mean_replays_frs[aclu], short_mean_replays_frs[aclu]) for aclu in long_mean_replays_frs.keys()}

In [ ]:
# 2023-09-05 - Migration Idea:
def _perform_update_pipeline(curr_active_pipeline):
	""" 2023-09-05 - Single example migration for Rate Remapping outputs to a new `RateRemappingResult` class: """
	## OLD Format:
	from pyphocorehelpers.DataStructure.dynamic_parameters import DynamicParameters
	curr_long_short_post_decoding = curr_active_pipeline.global_computation_results.computed_data['long_short_post_decoding']
	assert isinstance(curr_long_short_post_decoding, DynamicParameters)

	## Extract variables from results object:
	curr_long_short_rr = curr_long_short_post_decoding.rate_remapping
	assert isinstance(curr_long_short_rr, DynamicParameters)
	## Confirmed the old format. Extract the old results for conversion to new format:
	rate_remapping_df, high_remapping_cells_only = curr_long_short_rr.rr_df, curr_long_short_rr.high_only_rr_df

	## NEW Format:
	from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import RateRemappingResult
	rate_remapping_result = RateRemappingResult(rr_df=rate_remapping_df, high_only_rr_df=high_remapping_cells_only, considerable_remapping_threshold=0.7, is_global=True)

	curr_long_short_post_decoding.rate_remapping = rate_remapping_result # update the result
	print(f'updated')
	return curr_active_pipeline



In [ ]:
curr_active_pipeline.export_pipeline_to_h5()

In [ ]:
# curr_active_pipeline.save_pipeline()
curr_active_pipeline.save_global_computation_results()

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import batch_extended_computations

curr_active_pipeline.reload_default_computation_functions()

extended_computations_include_includelist = ['pf_computation', 'pfdt_computation', 'long_short_fr_indicies_analyses', 'jonathan_firing_rate_analysis', 'long_short_decoding_analyses', 'long_short_post_decoding', 
    '_perform_time_dependent_pf_sequential_surprise_computation',
     'long_short_rate_remapping',
     'long_short_inst_spike_rate_groups'
    ] # do only specifiedl , 'long_short_rate_remapping'
newly_computed_values = batch_extended_computations(curr_active_pipeline, include_includelist=extended_computations_include_includelist, include_global_functions=True, fail_on_exception=True, progress_print=True, force_recompute=False, debug_print=False)
newly_computed_values

In [ ]:
rate_remapping_df

In [ ]:
rate_remapping_df[['laps',	'replays',	'skew',	'max_axis_distance_from_center', 'distance_from_center', 'has_considerable_remapping']] # drops ['neuron_type', 'render_color']

In [ ]:
from neuropy.utils.mixins.HDF5_representable import HDF_Converter

file_path = 'output/test_rate_remapping.h5'

session_context = curr_active_pipeline.get_session_context() 
session_group_key: str = "/" + session_context.get_description(separator="/", include_property_names=False) # 'kdiba/gor01/one/2006-6-08_14-26-15'
session_uid: str = session_context.get_description(separator="|", include_property_names=False)
## Global Computations
a_global_computations_group_key: str = f"{session_group_key}/global_computations"
with tb.open_file(file_path, mode='w') as f:
    a_global_computations_group = f.create_group(session_group_key, 'global_computations', title='the result of computations that operate over many or all of the filters in the session.', createparents=True)


# rate_remapping_df
# rate_remapping_df = rate_remapping_df[['laps',	'replays',	'skew',	'max_axis_distance_from_center',	'distance_from_center', 	'has_considerable_remapping']]
# # active_context = kwargs.pop('active_context', None) # TODO: requiring that the caller pass active_context isn't optimal
# active_context = session_context
# rate_remapping_df = HDF_Converter.prepare_neuron_indexed_dataframe_for_hdf(rate_remapping_df, active_context=active_context, aclu_column_name=None)
# # rate_remapping_df.to_hdf(file_path, key=f'{key}/rate_remapping_df', format='table', data_columns=True)
curr_long_short_rr.to_hdf(file_path, key=f'{a_global_computations_group_key}/rate_remapping', active_context=curr_active_pipeline.get_session_context())

In [ ]:
# curr_active_pipeline.global_computation_results_pickle_path
# curr_active_pipeline.pickle_path
curr_active_pipeline.get_output_path()

In [ ]:
import ipywidgets as widgets
from IPython.display import display
from pyphoplacecellanalysis.GUI.IPyWidgets.pipeline_ipywidgets import interactive_pipeline_files


def fullwidth_path_widget(a_path):
	left_label = widgets.Label("session path:", layout=widgets.Layout(width='auto'))
	right_label = widgets.Label(a_path, layout=widgets.Layout(width='auto', flex='1'))
	hbox = widgets.HBox([left_label, right_label], layout=widgets.Layout(display='flex'))
	return hbox


session_path = str(curr_active_pipeline.get_output_path())
_session_path_widget = fullwidth_path_widget(a_path=session_path)
_button_executor = interactive_pipeline_files(curr_active_pipeline, defer_display=True)


_out_widget = widgets.VBox([_session_path_widget, _button_executor.root_widget])
display(_out_widget)


In [ ]:
from pyphoplacecellanalysis.PhoPositionalData.plotting.placefield import plot_1d_placecell_validations

# curr_active_pipeline.plotting_config
active_session_configuration_name = global_epoch_name
active_config = curr_active_pipeline.active_configs[active_session_configuration_name]

_out = plot_1d_placecell_validations(global_pf1D, active_config.plotting_config, modifier_string='lap_only', should_save=False)

In [ ]:
_out

In [ ]:
curr_active_pipeline.to_hdf('output/test_pipeline_fixed.h5', key='/')
# TypeError: objects of type ``list`` are not supported in this context, sorry; supported objects are: NumPy array, record or scalar
# for flattened_spike
# .infer_objects()

# curr_active_pipeline.export_pipeline_to_h5()
# self.to_hdf(file_path=hdf5_output_path, key="/")

In [ ]:
from neuropy.core.session.dataSession import DataSession

## Test saaving sessions
sess: DataSession = deepcopy(curr_active_pipeline.sess)
sess

In [ ]:
sess.to_hdf('output/test_sess.h5', key='/')

In [ ]:
sess.laps
# sess.ripples
sess.pbe

In [ ]:
curr_active_pipeline


In [ ]:
## Extract rr_* variables from rate_remapping_df
rr_aclus = rate_remapping_df.index.values
rr_laps, rr_replays, rr_skew, rr_neuron_type = [rate_remapping_df[n].values for n in ['laps', 'replays', 'skew', 'neuron_type']]


## Display:
# from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu
# n_debug_limit = 100
# fig, ax = plot_rr_aclu([str(aclu) for aclu in rr_aclus[:n_debug_limit]], rr_laps=rr_laps[:n_debug_limit], rr_replays=rr_replays[:n_debug_limit])

## Display Paginated multi-plot
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import plot_rr_aclu
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import RateRemappingPaginatedFigureController
active_identifying_session_ctx = curr_active_pipeline.sess.get_context()
_out_rr_pagination_controller = RateRemappingPaginatedFigureController.init_from_rr_data(rr_aclus, rr_laps, rr_replays, rr_neuron_type, max_subplots_per_page=30, a_name='TestRateRemappingPaginatedFigureController', active_context=active_identifying_session_ctx)
a_paginator = _out_rr_pagination_controller.plots_data.paginator

In [ ]:
from neuropy.analyses.time_dependent_placefields import PfND_TimeDependent, PlacefieldSnapshot

global_pf1D_dt: PfND_TimeDependent = global_results.pf1D_dt
global_pf2D_dt: PfND_TimeDependent = global_results.pf2D_dt


In [ ]:
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedOccupancyPlotter import TimeSynchronizedOccupancyPlotter
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.TimeSynchronizedPlacefieldsPlotter import TimeSynchronizedPlacefieldsPlotter
from pyphoplacecellanalysis.Pho2D.PyQtPlots.TimeSynchronizedPlotters.Mixins.helpers import build_combined_time_synchronized_plotters_window

active_pf_2D_dt = global_pf2D_dt
active_pf_2D_dt.reset()
active_pf_2D_dt.update(t=500.0, start_relative_t=True)
all_plotters, root_dockAreaWindow, app = build_combined_time_synchronized_plotters_window(active_pf_2D_dt, fixed_window_duration = 15.0)
controlling_widget, curr_sync_occupancy_plotter, curr_placefields_plotter = all_plotters # end up in some sort of `linkedViewChanged` loop

In [ ]:
controlling_widget

In [ ]:
# Create a new `SpikeRaster2D` instance using `_display_spike_raster_pyqtplot_2D` and capture its outputs:
_out = curr_active_pipeline.plot._display_spike_rasters_pyqtplot_2D()

In [ ]:
from pyphocorehelpers.gui.Qt.TopLevelWindowHelper import TopLevelWindowHelper
import pyphoplacecellanalysis.External.pyqtgraph as pg # Used to get the app for TopLevelWindowHelper.top_level_windows
## For searching with `TopLevelWindowHelper.all_widgets(...)`:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster import Spike2DRaster
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike3DRaster import Spike3DRaster
from pyphoplacecellanalysis.GUI.Qt.SpikeRasterWindows.Spike3DRasterWindowWidget import Spike3DRasterWindowWidget

found_spike_raster_windows = TopLevelWindowHelper.all_widgets(pg.mkQApp(), searchType=Spike3DRasterWindowWidget)
# assert len(found_spike_raster_windows) == 1, f"found {len(found_spike_raster_windows)} Spike3DRasterWindowWidget windows using TopLevelWindowHelper.all_widgets(...) but require exactly one."
spike_raster_window = found_spike_raster_windows[-1]
# Extras:
active_2d_plot = spike_raster_window.spike_raster_plt_2d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>
active_3d_plot = spike_raster_window.spike_raster_plt_3d # <pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.SpikeRasterWidgets.Spike2DRaster.Spike2DRaster at 0x196c7244280>

In [ ]:
from pyphoplacecellanalysis.General.Mixins.SpikesRenderingBaseMixin import SpikeEmphasisState

active_pf_2D = global_pf2D
## Example 1: De-emphasize spikes excluded from the placefield calculations:
is_spike_included_in_pf = np.isin(spike_raster_window.spike_raster_plt_2d.spikes_df.index, active_pf_2D.filtered_spikes_df.index)
spike_raster_window.spike_raster_plt_2d.update_spike_emphasis(np.logical_not(is_spike_included_in_pf), SpikeEmphasisState.Deemphasized)


## 📣 Programmatically adding several epoch rectangles by calling the addRenderable context menu functions all at once for SpikeRaster2D

In [ ]:
add_renderables_menu = active_2d_plot.ui.menus.custom_context_menus.add_renderables[0].programmatic_actions_dict
menu_commands = ['AddTimeIntervals.PBEs', 'AddTimeIntervals.Ripples', 'AddTimeIntervals.Replays', 'AddTimeIntervals.Laps', 'AddTimeIntervals.Session.Epochs']
for a_command in menu_commands:
    add_renderables_menu[a_command].trigger()

In [ ]:
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.Mixins.RenderTimeEpochs.Specific2DRenderTimeEpochs import General2DRenderTimeEpochs, Ripples_2DRenderTimeEpochs, inline_mkColor
## Inline Concise: Position Replays, PBEs, and Ripples all below the scatter:
# active_2d_plot.interval_datasources.Replays.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.PBEs.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.Ripples.update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
# active_2d_plot.interval_datasources.SessionEpochs .update_visualization_properties(lambda active_df, **kwargs: General2DRenderTimeEpochs._update_df_visualization_columns(active_df, y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5), **kwargs)) ## Fully inline
epochs_update_dict = {
    'Replays':dict(y_location=-10.0, height=7.5, pen_color=inline_mkColor('orange', 0.8), brush_color=inline_mkColor('orange', 0.5)),
    'PBEs':dict(y_location=-2.0, height=1.5, pen_color=inline_mkColor('pink', 0.8), brush_color=inline_mkColor('pink', 0.5)),
    'Ripples':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
    'SessionEpochs ':dict(y_location=-12.0, height=1.5, pen_color=inline_mkColor('cyan', 0.8), brush_color=inline_mkColor('cyan', 0.5)),
}
active_2d_plot.update_rendered_intervals_visualization_properties(epochs_update_dict)


In [ ]:
rendered_interval_keys = list(interval_info.keys())
desired_interval_height_ratios = [2.0, 2.0, 1.0, 0.1, 1.0, 1.0, 1.0] # ratio of heights to each interval
required_vertical_offsets, required_interval_heights = EpochRenderingMixin.build_stacked_epoch_layout(desired_interval_height_ratios, epoch_render_stack_height=20.0, interval_stack_location='below')
stacked_epoch_layout_dict = {interval_key:dict(y_location=y_location, height=height) for interval_key, y_location, height in zip(rendered_interval_keys, required_vertical_offsets, required_interval_heights)} # Build a stacked_epoch_layout_dict to update the display
active_2d_plot.update_rendered_intervals_visualization_properties(stacked_epoch_layout_dict)

In [ ]:
is_spike_included_in_pf

In [ ]:
active_pf_2D_dt.update(t=1000.0, start_relative_t=True)

In [ ]:

curr_sync_occupancy_plotter = TimeSynchronizedOccupancyPlotter(global_pf2D_dt)
curr_sync_occupancy_plotter.show()

In [ ]:
global_pf2D_dt.historical_snapshots

In [ ]:
global_pf2D_dt.batch_snapshotting

In [ ]:
curr_sync_placefield_plotter = TimeSynchronizedPlacefieldsPlotter(active_time_dependent_placefields2D)
curr_sync_placefield_plotter.show()

In [ ]:
# Make sure pickling is okay with pf1D_dt
global_pf1D_dt.to_hdf('output/test_global_pf1D_dt.h5', key='global_pf1D_dt')

In [ ]:
override_pickle_path: Path = Path('output/test_pipeline.pkl').resolve()
override_pickle_path

In [ ]:
curr_active_pipeline.save_pipeline(active_pickle_filename=None, override_pickle_path=override_pickle_path)

In [ ]:
## TODO: test loading from this path:
override_pickle_path: Path = Path('output/test_pipeline.pkl').resolve()
override_pickle_path

_loaded_pipeline = NeuropyPipeline.try_init_from_saved_pickle_or_reload_if_needed('kdiba', known_type_properties=KDibaOldDataSessionFormatRegisteredClass.get_known_data_session_type_properties(), override_basepath=override_pickle_path.parent, active_pickle_filename='test_pipeline.pkl', skip_save_on_initial_load=True, progress_print=True, debug_print=False)


In [ ]:
curr_active_pipeline.save_pipeline()

In [ ]:
active_extended_stats = curr_active_pipeline.computation_results['maze'].computed_data['extended_stats']
active_extended_stats


In [ ]:
from pyphoplacecellanalysis.External.pyqtgraph import QtWidgets, QtCore, QtGui
from pyphoplacecellanalysis.GUI.Qt.MainApplicationWindows.LauncherWidget.LauncherWidget import LauncherWidget

widget = LauncherWidget()
treeWidget = widget.mainTreeWidget # QTreeWidget
widget.build_for_pipeline(curr_active_pipeline=curr_active_pipeline)
widget.show()

In [ ]:
active_relative_entropy_results = active_extended_stats['relative_entropy_analyses']
post_update_times = active_relative_entropy_results['post_update_times']
snapshot_differences_result_dict = active_relative_entropy_results['snapshot_differences_result_dict']
time_intervals = active_relative_entropy_results['time_intervals']
long_short_rel_entr_curves_frames = active_relative_entropy_results['long_short_rel_entr_curves_frames']
short_long_rel_entr_curves_frames = active_relative_entropy_results['short_long_rel_entr_curves_frames']
flat_relative_entropy_results = active_relative_entropy_results['flat_relative_entropy_results']
flat_jensen_shannon_distance_results = active_relative_entropy_results['flat_jensen_shannon_distance_results']

In [ ]:
global_pf1D_dt.plot_ratemaps_1D()

In [ ]:
curr_active_pipeline.global_computation_results.

In [ ]:
# Compute with the new computation config:
computation_functions_name_includelist=['_perform_baseline_placefield_computation', '_perform_time_dependent_placefield_computation', '_perform_extended_statistics_computation',
									'_perform_position_decoding_computation', 
									'_perform_firing_rate_trends_computation',
									'_perform_pf_find_ratemap_peaks_computation',
									'_perform_time_dependent_pf_sequential_surprise_computation'
									'_perform_two_step_position_decoding_computation',
									# '_perform_recursive_latent_placefield_decoding'
								]  # '_perform_pf_find_ratemap_peaks_peak_prominence2d_computation'

# computation_functions_name_includelist=['_perform_baseline_placefield_computation']
curr_active_pipeline.perform_computations(computation_functions_name_includelist=computation_functions_name_includelist, computation_functions_name_excludelist=None, fail_on_exception=True, debug_print=False, overwrite_extant_results=True) #, overwrite_extant_results=False  ], fail_on_exception=True, debug_print=False)

In [ ]:
# global_pf1D.included_neuron_IDs
included_neuron_IDs = list(long_pf1D.included_neuron_IDs[0])
print(included_neuron_IDs)
included_neuron_IDs.index(4)
long_pf1D.long
# long_pf1D.included_neuron_IDs[3]


In [ ]:
long_pf1D.neuron_extended_ids

In [ ]:
NeuronExtendedIdentityTuple(shank=10, cluster=10, id=87),

In [ ]:
list(long_pf1D.included_neuron_IDs).index(87)

In [ ]:
long_pf1D.plot_all(87)

In [ ]:
long_pf1D.plotRaw_v_time(87)

In [ ]:


curr_active_pipeline.display('_display_batch_pho_jonathan_replay_firing_rate_comparison')

In [ ]:
matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=False)

In [ ]:
from typing import Optional
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from neuropy.core.neurons import NeuronType
from neuropy.utils.mixins.HDF5_representable import HDF_Converter

# Generate the unique aclus
_neuron_replay_stats_df = deepcopy(jonathan_firing_rate_analysis_result.neuron_replay_stats_df)
_neuron_replay_stats_df

# _neuron_replay_stats_df = prepare_neuron_indexed_dataframe_for_hdf(_neuron_replay_stats_df, active_context=curr_active_pipeline.get_session_context(), aclu_column_name=None)


_neuron_replay_stats_df = HDF_Converter.prepare_neuron_indexed_dataframe_for_hdf(_neuron_replay_stats_df, active_context=curr_active_pipeline.get_session_context(), aclu_column_name=None)



# active_context = curr_active_pipeline.get_session_context()
# sess_specific_aclus = list(_neuron_replay_stats_df.index.to_numpy())
# session_ctxt_key:str = active_context.get_description(separator='|', subset_includelist=IdentifyingContext._get_session_context_keys())

# # Adds column columns=['neuron_uid', 'session_uid', 'aclu']
# _neuron_replay_stats_df['aclu'] = sess_specific_aclus # _neuron_replay_stats_df.index.to_numpy() # add explicit 'aclu' column from index
# _neuron_replay_stats_df['session_uid'] = session_ctxt_key # add fixed 'session_uid' column 
# _neuron_replay_stats_df['neuron_uid'] = [f"{session_ctxt_key}|{aclu}" for aclu in sess_specific_aclus]
_neuron_replay_stats_df



In [ ]:
curr_active_pipeline.reload_default_display_functions()

In [ ]:

# _out = curr_active_pipeline.last_added_display_output
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
_out = curr_active_pipeline.display('_display_long_short_expected_v_observed_firing_rate', max_num_rows=10)
# a_fig = _out.figures[0]
# a_fig.show()
# .show()
# plt.show()


In [ ]:
expected_v_observed_result

In [ ]:
rate_remapping_df

In [ ]:
## Use the `long_short_decoding_analyses` global result to access `long_results_obj.active_filter_epochs`:
curr_long_short_decoding_analyses = curr_active_pipeline.global_computation_results.computed_data['long_short_leave_one_out_decoding_analysis']
long_one_step_decoder_1D, short_one_step_decoder_1D, long_replays, short_replays, global_replays, long_shared_aclus_only_decoder, short_shared_aclus_only_decoder, shared_aclus, long_short_pf_neurons_diff, n_neurons, long_results_obj, short_results_obj, is_global = curr_long_short_decoding_analyses.long_decoder, curr_long_short_decoding_analyses.short_decoder, curr_long_short_decoding_analyses.long_replays, curr_long_short_decoding_analyses.short_replays, curr_long_short_decoding_analyses.global_replays, curr_long_short_decoding_analyses.long_shared_aclus_only_decoder, curr_long_short_decoding_analyses.short_shared_aclus_only_decoder, curr_long_short_decoding_analyses.shared_aclus, curr_long_short_decoding_analyses.long_short_pf_neurons_diff, curr_long_short_decoding_analyses.n_neurons, curr_long_short_decoding_analyses.long_results_obj, curr_long_short_decoding_analyses.short_results_obj, curr_long_short_decoding_analyses.is_global


In [ ]:
# Ratemap
# ratemap = deepcopy(long_pf1D.ratemap)

ratemap: Ratemap = deepcopy(long_pf2D.ratemap)
ratemap

In [ ]:
import h5py
from neuropy.core.ratemap import Ratemap


# ratemap.to_hdf('output/test_ratemap.h5', key='ratemap', file_mode='w')

In [ ]:
# Plot long|short firing rate index:
long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
x_frs_index, y_frs_index = long_short_fr_indicies_analysis_results['x_frs_index'], long_short_fr_indicies_analysis_results['y_frs_index'] # use the all_results_dict as the computed data value
active_context = long_short_fr_indicies_analysis_results['active_context']

# final_context = active_context.adding_context('display_fn', display_fn_name='display_long_short_laps')
# fig = _plot_long_short_firing_rate_indicies(x_frs_index, y_frs_index, final_context, debug_print=debug_print)

In [ ]:
long_short_fr_indicies_analysis_results

In [ ]:
h5_test_file_path = Path(f'output/test_long_short_fr_indicies_analysis_results.h5').resolve()
h5_test_file_path
long_short_fr_indicies_analysis_results.to_hdf(h5_test_file_path, key='long_short_fr_indicies_analysis_results')

# jonathan_firing_rate_analysis_result.is_hdf_serializable()

In [ ]:

h5_test_file_path = Path(f'output/test_long_short_decoding_analyses.h5').resolve()
h5_test_file_path
curr_long_short_decoding_analyses.to_hdf(h5_test_file_path, key='long_short_decoding_analyses')

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import LeaveOneOutDecodingResult, LeaveOneOutDecodingAnalysisResult, TimebinnedNeuronActivity, _convert_dict_to_hdf_attrs_fn

# curr_long_short_decoding_analyses.long_results_obj.new_result


In [ ]:

print(f'type(curr_long_short_decoding_analyses.long_results_obj): {type(curr_long_short_decoding_analyses.long_results_obj)}')
h5_test_file_path = Path(f'output/test_LeaveOneOutDecodingAnalysisResult.h5').resolve()
curr_long_short_decoding_analyses.long_results_obj.to_hdf(h5_test_file_path, key='long_results_obj')


In [ ]:
curr_long_short_decoding_analyses.short_results_obj.to_hdf(h5_test_file_path, key='short_results_obj')

In [ ]:


_obj = deepcopy(curr_long_short_decoding_analyses.long_results_obj.timebinned_neuron_info)

In [ ]:
# np.hstack((_obj.active_IDXs, _obj.inactive_IDXs))

[np.hstack((a_v, i_v)) for a_v, i_v in zip(_obj.active_IDXs, _obj.inactive_IDXs)]

In [ ]:
# TimebinnedNeuronActivity
h5_test_file_path = Path(f'output/test_TimebinnedNeuronActivity.h5').resolve()
curr_long_short_decoding_analyses.long_results_obj.timebinned_neuron_info.to_hdf(h5_test_file_path, key='timebinned_neuron_info')

In [ ]:
print(f'type(curr_long_short_decoding_analyses.long_results_obj.new_result): {type(curr_long_short_decoding_analyses.long_results_obj.new_result)}')
h5_test_file_path = Path(f'output/test_LeaveOneOutDecodingResult.h5').resolve()
curr_long_short_decoding_analyses.long_results_obj.new_result.to_hdf(h5_test_file_path, key='new_result', file_mode='r+')

In [ ]:
curr_long_short_decoding_analyses.short_results_obj

In [ ]:
jonathan_firing_rate_analysis_result.rdf.rdf


In [ ]:
from pyphoplacecellanalysis.General.Mixins.CrossComputationComparisonHelpers import SplitPartitionMembership
from neuropy.core.neurons import NeuronType

jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations import JonathanFiringRateAnalysisResult

h5_test_file_path = Path(f'output/test_jonathan_firing_rate_analysis_result.h5').resolve()
h5_test_file_path
jonathan_firing_rate_analysis_result.to_hdf(h5_test_file_path, key='jonathan_firing_rate_analysis_result')

# jonathan_firing_rate_analysis_result.is_hdf_serializable()

In [ ]:
aclu_to_idx = jonathan_firing_rate_analysis_result.rdf.aclu_to_idx
aclu_to_idx_df: pd.DataFrame = pd.DataFrame({'aclu': list(aclu_to_idx.keys()), 'fragile_linear_idx': list(aclu_to_idx.values())})

In [ ]:


# I wanna look at the suprise metrics again please.

# can align each of the sessions (across days) to the track transition point (the "Delta") as the zero.

# print_keys_if_possible("global_computation_results", curr_active_pipeline.global_computation_results, max_depth=2)

# computed_data: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── jonathan_firing_rate_analysis: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── long_short_fr_indicies_analysis: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters
# 		│   ├── long_short_leave_one_out_decoding_analysis: pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.LongShortTrackComputations.LeaveOneOutDecodingAnalysis
# 		│   ├── long_short_post_decoding: pyphocorehelpers.DataStructure.dynamic_parameters.DynamicParameters


print_keys_if_possible("global_computation_results", curr_active_pipeline.global_computation_results.computed_data, max_depth=3)

# jonathan_firing_rate_analysis_result.rdf

# long_short_leave_one_out_decoding_analysis

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_fr_indicies_analysis

In [ ]:
from SCRATCH.PhoObjectBrowser_JupyterWidget import ObjectBrowser

browser = ObjectBrowser(object_to_browse=curr_active_pipeline.global_computation_results.computed_data)
browser.show()

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding.expected_v_observed_result
# curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding

In [ ]:
curr_active_pipeline.global_computation_results.computed_data

In [ ]:
# Iterative partition
curr_active_pipeline.global_computation_results.computed_data

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding.expected_v_observed_result

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding

In [ ]:
curr_active_pipeline.global_computation_results.computed_data.long_short_post_decoding

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, InstantaneousFiringRatesDataframeAccessor, InstantaneousSpikeRateGroupsComputation, trackMembershipTypesEnum, trackExclusiveToMembershipTypeDict, trackExclusiveToMembershipTypeReverseDict

# ## Specify the output file:
# common_file_path = Path('output/test_across_session_scatter_plot_new.h5')
# print(f'common_file_path: {common_file_path}')
# InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

In [ ]:
import seaborn as sns
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor, AcrossSessionsVisualizations
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import SingleBarResult

common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
print(f'common_file_path: {common_file_path}')

# InstantaneousSpikeRateGroupsComputation, : pd.DataFrame
_shell_obj, loaded_result_df = InstantaneousFiringRatesDataframeAccessor.load_and_prepare_for_plot(common_file_path)
# Perform the actual plotting:
AcrossSessionsVisualizations.across_sessions_bar_graphs(_shell_obj, num_sessions=13, save_figure=False, enable_tiny_point_labels=False, enable_hover_labels=True)

In [ ]:
from datetime import datetime, timedelta

maximum_timedelta: timedelta = timedelta(1, 0, 0) # 1 Day maximum time
delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
print(f'delta_since_last_compute: {delta_since_last_compute}')
needs_recompute: bool = delta_since_last_compute > maximum_timedelta
print(f'\tneeds_recompute: {needs_recompute}')

In [ ]:
## Serialization of preprocessing parameters test
file_path = 'output/preprocessing_parameters.h5'

with h5py.File(file_path, "w") as f:
	preprocessing_group = f.create_group("preprocessing_parameters")

	# Serialize epoch_estimation_parameters
	epoch_estimation_group = preprocessing_group.create_group("epoch_estimation_parameters")

	laps_group = epoch_estimation_group.create_group("laps")
	laps_group.attrs["N"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["laps"]["N"]
	laps_group.attrs["should_backup_extant_laps_obj"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["laps"]["should_backup_extant_laps_obj"]

	PBEs_group = epoch_estimation_group.create_group("PBEs")
	PBEs_group.attrs["thresh"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["thresh"]
	PBEs_group.attrs["min_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["min_dur"]
	PBEs_group.attrs["merge_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["merge_dur"]
	PBEs_group.attrs["max_dur"] = preprocessing_parameters_dict["epoch_estimation_parameters"]["PBEs"]["max_dur"]

	replays_group = epoch_estimation_group.create_group("replays")
	replay_data = preprocessing_parameters_dict["epoch_estimation_parameters"]["replays"]
	# replay_dataframe = pd.DataFrame(replay_data)
	# replay_data.to_hdf(f, "/preprocessing_parameters/epoch_estimation_parameters/replays")
	## TODO: add the data here using Epoch's .to_hdf
	
	# Check for "None" values before setting attributes
	def check_and_set(key, value):
		if value is not None:
			replays_group.attrs[key] = value

	# Set attributes if not "None", otherwise skip writing
	check_and_set("min_epoch_included_duration", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_epoch_included_duration"))
	check_and_set("max_epoch_included_duration", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("max_epoch_included_duration"))
	check_and_set("maximum_speed_thresh", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("maximum_speed_thresh"))
	check_and_set("min_inclusion_fr_active_thresh", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_inclusion_fr_active_thresh"))
	check_and_set("min_num_unique_aclu_inclusions", preprocessing_parameters_dict["epoch_estimation_parameters"].get('replays', {}).get("min_num_unique_aclu_inclusions"))

In [ ]:
enable_default_neptune_plots = False
# enable_default_neptune_plots = True

## To file only:
with matplotlib_file_only():
	# Perform non-interactive Matplotlib operations with 'AGG' backend
	main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=enable_default_neptune_plots, save_figures_only=True, save_figure=True)

## Clear the Programmatically open figures:
plt.close('all') # this takes care of the matplotlib-backed figures.
curr_active_pipeline.clear_display_outputs()
curr_active_pipeline.clear_registered_output_files()

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_out_figures = main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=False, save_figures_only=False, save_figure=True)

# Common Display Setup

In [ ]:
import matplotlib
# configure backend here
matplotlib.use('Qt5Agg')
# backend_qt5agg
# %matplotlib qt5
# %matplotlib qt
# matplotlib.use('AGG') # non-interactive backend ## 2022-08-16 - Surprisingly this works to make the matplotlib figures render only to .png file, not appear on the screen!

import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()
# mpl.rcParams['toolbar'] = 'None' # disable toolbars

# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')

# Figure 1) pf1D Ratemaps, Active set, etc

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.SpikeRasters import plot_multiple_raster_plot, plot_raster_plot
from pyphoplacecellanalysis.Pho2D.matplotlib.visualize_heatmap import visualize_heatmap_pyqtgraph # used in `plot_kourosh_activity_style_figure`
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PAPER_FIGURE_figure_1_full, PAPER_FIGURE_figure_1_add_replay_epoch_rasters

pf1d_compare_graphics, (example_epoch_rasters_L, example_epoch_rasters_S), example_stacked_epoch_graphics = PAPER_FIGURE_figure_1_full(curr_active_pipeline) # did not display the pf1

In [ ]:
rdf = jonathan_firing_rate_analysis_result.rdf.rdf
rdf


In [ ]:

['start']


# Any change in duration over the recording session duration?
rdf['duration']

# rdf.plot.scatter('start', 'duration')

# rdf.plot.scatter('duration', 'num_neuron_participating') # strong positive trend

# rdf.plot.scatter('start', 'num_neuron_participating') # num neurons participating seems to increase over time, especially after the short track is introduced. Contrary to my hypothesis.

rdf.plot.scatter('start', 'num_short_only_neuron_participating')




# ['num_neuron_participating']

In [ ]:
curr_active_pipeline.reload_default_display_functions()
a_plot_obj = curr_active_pipeline.plot
a_plot_obj._display_spike_rasters_pyqtplot_2D()
# '_display_spike_rasters_pyqtplot_2D': ' Plots a standalone 2D raster plot\n        ',
# '_display_spike_rasters_pyqtplot_3D': ' Plots a standalone 3D raster plot with independent/standalone controls built-in\n        ',
# '_display_spike_rasters_pyqtplot_3D_with_2D_controls': ' Plots a standalone 3D raster plot (via pyqtgraph) with a separate 2D raster plot as the window with which you can adjust the viewed window. \n        ',
# '_display_spike_rasters_vedo_3D': ' Plots a standalone 3D raster plot with independent/standalone controls built-in\n        ',
# '_display_spike_rasters_vedo_3D_with_2D_controls': ' Plots a standalone 3D raster plot (via Vedo) with a separate 2D raster plot as the window with which you can adjust the viewed window. \n        ',
# '_display_spike_rasters_window': ' Displays a Spike3DRasterWindowWidget with a configurable set of raster widgets and controls in it.\n        ',type

In [ ]:
type(a_plot_obj)

In [ ]:
curr_active_pipeline.registered_display_function_docs_dict

In [ ]:
pf1d_compare_fig_L, pf1d_compare_fig_S = pf1d_compare_graphics.figures


In [ ]:
pf1d_compare_fig_L.show()

In [ ]:
plt.show()

In [ ]:
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import build_shared_sorted_neuronIDs

ratemap = long_pf1D.ratemap
included_unit_neuron_IDs = EITHER_subset.track_exclusive_aclus
rediculous_final_sorted_all_included_neuron_ID, rediculous_final_sorted_all_included_pfmap = build_shared_sorted_neuronIDs(ratemap, included_unit_neuron_IDs, sort_ind=new_all_aclus_sort_indicies.copy())

In [ ]:
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(rediculous_final_sorted_all_included_pfmap, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
active_curves_sorted = long_pf1D.ratemap.normalized_tuning_curves[is_included][included_new_all_aclus_sort_indicies]
heatmap_pf1D_win, heatmap_pf1D_img = visualize_heatmap_pyqtgraph(active_curves_sorted, show_yticks=False, title=f"pf1D Sorted Visualization", defer_show=True)

In [ ]:
from pyphoplacecellanalysis.Pho2D.stacked_epoch_slices import DecodedEpochSlicesPaginatedFigureController

## Stacked Epoch Plot
example_stacked_epoch_graphics = curr_active_pipeline.display('_display_long_and_short_stacked_epoch_slices', defer_render=False, save_figure=True)
pagination_controller_L, pagination_controller_S = example_stacked_epoch_graphics.plot_data['controllers']
ax_L, ax_S = example_stacked_epoch_graphics.axes
final_figure_context_L, final_context_S = example_stacked_epoch_graphics.context

In [ ]:
# Set the symbol properties
# symbol = pg.mkPen('w', width=1)  # Pen for the lines
size = 1.0  # Fixed x-width
symbol_brush = None  # No brush for the symbol (transparent fill)

# override_scatter_plot_kwargs = dict(pxMode=False, symbol='|', size=size, pen={'color': 'w', 'width': 1.0}) # , brush=symbol_brush

override_scatter_plot_kwargs = dict(pxMode=False, symbol='arrow_up', size=1.0, pen={'color': 'w', 'width': 1.0}, hoverable=True) # 

In [ ]:
filter_epoch_spikes_df_L.spikes.rebuild_fragile_linear_neuron_IDXs();
filter_epoch_spikes_df_S.spikes.rebuild_fragile_linear_neuron_IDXs();

In [ ]:
app_L, win_L, plots_L, plots_data_L = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
										epoch_id_key_name='replay_epoch_id', scatter_app_name="Long Decoded Example Replays")

In [ ]:
app_S, win_S, plots_S, plots_data_S = plot_multiple_raster_plot(epochs_df_S, filter_epoch_spikes_df_S, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                 epoch_id_key_name='replay_epoch_id', scatter_app_name="Short Decoded Example Replays")

In [ ]:
# Test single `plot_raster_plot` calls
an_epoch = list(epochs_df_L.itertuples())[0]
an_epoch_spikes_df = filter_epoch_spikes_df_L[filter_epoch_spikes_df_L['replay_epoch_id'] == an_epoch.Index]

_out_single_raster_plot = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test1")
_out_single_raster_plot2 = plot_raster_plot(an_epoch_spikes_df, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs, scatter_app_name="test2")

In [ ]:
app_alt, win_alt, plots_alt, plots_data_alt = plot_multiple_raster_plot(epochs_df_L, filter_epoch_spikes_df_L, included_neuron_ids=EITHER_subset.track_exclusive_aclus, unit_sort_order=None, unit_colors_list=unit_colors_list, scatter_plot_kwargs=override_scatter_plot_kwargs,
                                                                         epoch_id_key_name='replay_epoch_id', scatter_app_name="ALT Long Decoded Example Replays")

### Testing `plot_kourosh_activity_style_figure` for debugging:

In [ ]:
from pyphoplacecellanalysis.Analysis.Decoder.decoder_result import plot_kourosh_activity_style_figure
from pyphoplacecellanalysis.GUI.PyQtPlot.Widgets.helpers import _helper_make_scatterplot_clickable

plot_aclus = EITHER_subset.track_exclusive_aclus.copy()
# plot_aclus = EITHER_subset.track_exclusive_aclus[new_all_aclus_sort_indicies].copy()
_out_A = plot_kourosh_activity_style_figure(long_results_obj, long_session, plot_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=13, callout_epoch_IDXs=None, skip_rendering_callouts=False)

In [ ]:
app, win, plots, plots_data = _out_A
# plots

In [ ]:
#TODO 2023-06-27 10:42: - [ ] Desperitely need a class that "explodes" the important variables and their types out of a DynamicParameters (dict-like) or other object.


In [ ]:
_out_n = plot_kourosh_activity_style_figure(long_results_obj, long_session, EITHER_subset.track_exclusive_aclus, unit_sort_order=new_all_aclus_sort_indicies, epoch_idx=49, callout_epoch_IDXs=np.arange(6), skip_rendering_callouts=False)

# 2023-07-14 - LxC and SxC PhoJonathanSession plots

In [ ]:
from pyphoplacecellanalysis.General.Batch.NonInteractiveProcessing import BatchPhoJonathanFiguresHelper

## Get global 'jonathan_firing_rate_analysis' results:
curr_jonathan_firing_rate_analysis = curr_active_pipeline.global_computation_results.computed_data['jonathan_firing_rate_analysis']
neuron_replay_stats_df, rdf, aclu_to_idx, irdf = curr_jonathan_firing_rate_analysis.neuron_replay_stats_df, curr_jonathan_firing_rate_analysis.rdf.rdf, curr_jonathan_firing_rate_analysis.rdf.aclu_to_idx, curr_jonathan_firing_rate_analysis.irdf.irdf

# with VizTracer(output_file=f"viztracer_display_BatchPhoJonathanFiguresHelper_PlusPDF_20.json", min_duration=200, tracer_entries=3000000, ignore_frozen=True) as tracer:
active_out_figures_dict = BatchPhoJonathanFiguresHelper.run(curr_active_pipeline, neuron_replay_stats_df, included_unit_neuron_IDs=XOR_subset.track_exclusive_aclus, n_max_page_rows=20, write_vector_format=False, write_png=True) # active_out_figures_dict: {IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'long_only', '(12,21,48)')>: <Figure size 1920x660 with 12 Axes>, IdentifyingContext<('kdiba', 'gor01', 'two', '2006-6-07_16-40-19', 'BatchPhoJonathanReplayFRC', 'short_only', '(18,19,65)')>: <Figure size 1920x660 with 12 Axes>}
print(f'active_out_figures_dict: {active_out_figures_dict}')


In [ ]:
# curr_active_pipeline.display(display_function='_display_2d_placefield_result_plot_ratemaps_2D', active_session_configuration_context=long_epoch_context)


# curr_active_pipeline.display(display_function='_display_placemaps_pyqtplot_2D', active_session_configuration_context=long_epoch_context)
curr_active_pipeline.display(display_function='_display_1d_placefields', active_session_configuration_context=long_epoch_context)
# curr_active_pipeline.display(display_function='_display_2d_placefield_occupancy', active_session_configuration_context=long_epoch_context)





In [ ]:
fig = plt.figure() # new figure to hold the result
# can show the figures by looping through and calling
for a_ctxt, a_fig in active_out_figures_dict.items():
    print(f'showing: {a_ctxt}')
    a_fig.show()
    


In [ ]:

curr_active_pipeline.registered_output_files_list

# Figure 2) Firing Rate Bar Graphs

In [ ]:
# Instantaneous versions:
from pyphocorehelpers.mixins.serialized import SerializedAttributesAllowBlockSpecifyingClass
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends

_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_fig_2.compute(curr_active_pipeline=curr_active_pipeline, active_context=curr_active_pipeline.sess.get_context())

_out_fig_2_figs = {}

restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_out_fig_2_figs = _out_fig_2.display(defer_show=True, save_figure=True, active_context=curr_active_pipeline.sess.get_context())

## Extract for debugging/checking:
_out_inst_fr_comps = _out_fig_2.computation_result
LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus = _out_inst_fr_comps.LxC_ReplayDeltaMinus, _out_inst_fr_comps.LxC_ReplayDeltaPlus, _out_inst_fr_comps.SxC_ReplayDeltaMinus, _out_inst_fr_comps.SxC_ReplayDeltaPlus
LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus = _out_inst_fr_comps.LxC_ThetaDeltaMinus, _out_inst_fr_comps.LxC_ThetaDeltaPlus, _out_inst_fr_comps.SxC_ThetaDeltaMinus, _out_inst_fr_comps.SxC_ThetaDeltaPlus
# LxC_ThetaDeltaMinus # cell_agg_inst_fr_list


In [ ]:
_out_inst_fr_comps = InstantaneousSpikeRateGroupsComputation(instantaneous_time_bin_size_seconds=0.01) # 10ms
_out_inst_fr_comps.compute(curr_active_pipeline=curr_active_pipeline, active_context=curr_active_pipeline.sess.get_context())
LxC_ReplayDeltaMinus, LxC_ReplayDeltaPlus, SxC_ReplayDeltaMinus, SxC_ReplayDeltaPlus = _out_inst_fr_comps.LxC_ReplayDeltaMinus, _out_inst_fr_comps.LxC_ReplayDeltaPlus, _out_inst_fr_comps.SxC_ReplayDeltaMinus, _out_inst_fr_comps.SxC_ReplayDeltaPlus
LxC_ThetaDeltaMinus, LxC_ThetaDeltaPlus, SxC_ThetaDeltaMinus, SxC_ThetaDeltaPlus = _out_inst_fr_comps.LxC_ThetaDeltaMinus, _out_inst_fr_comps.LxC_ThetaDeltaPlus, _out_inst_fr_comps.SxC_ThetaDeltaMinus, _out_inst_fr_comps.SxC_ThetaDeltaPlus

In [ ]:
inst_fr_output_filename = 'long_short_inst_firing_rates.pkl'
inst_fr_output_path = curr_active_pipeline.get_output_path().joinpath(inst_fr_output_filename).resolve()
inst_fr_output_path

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData

# saveData(inst_fr_output_path, _out_fig_2)
saveData(inst_fr_output_path, _out_fig_2.to_dict())


In [ ]:
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends


srt1 = PaperFigureTwo(**loadData(inst_fr_output_path))


In [ ]:
long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]] # only uses global_session
(epochs_df_L, epochs_df_S), (filter_epoch_spikes_df_L, filter_epoch_spikes_df_S), (good_example_epoch_indicies_L, good_example_epoch_indicies_S), (short_exclusive, long_exclusive, BOTH_subset, EITHER_subset, XOR_subset, NEITHER_subset), new_all_aclus_sort_indicies, assigning_epochs_obj = PAPER_FIGURE_figure_1_add_replay_epoch_rasters(curr_active_pipeline)

long_short_fr_indicies_analysis_results = curr_active_pipeline.global_computation_results.computed_data['long_short_fr_indicies_analysis']
long_laps, long_replays, short_laps, short_replays, global_laps, global_replays = [long_short_fr_indicies_analysis_results[k] for k in ['long_laps', 'long_replays', 'short_laps', 'short_replays', 'global_laps', 'global_replays']]


In [ ]:
jonathan_firing_rate_analysis_result.neuron_replay_stats_df

In [ ]:
long_exclusive.track_exclusive_df

In [ ]:
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df['lap_delta_plus_inst_fr'] = LxC_ThetaDeltaPlus.cell_agg_inst_fr_list # (n_cells, )
long_exclusive.track_exclusive_df


In [ ]:
# d = LxC_ThetaDeltaMinus.epoch_agg_inst_fr_list # (n_epochs, n_cells)
# d = d[:,1]
d = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )
d
long_exclusive.track_exclusive_df['lap_delta_minus_inst_fr'] = LxC_ThetaDeltaMinus.cell_agg_inst_fr_list # (n_cells, )


In [ ]:
## Drop out the very low (inactive) Epochs... I guess? But we want it to be influenced by the inactive epochs.
curr_active_pipeline.	

In [ ]:

len(LxC_ThetaDeltaMinus.inst_fr_signals_list) # n_epochs
d = LxC_ThetaDeltaMinus.inst_fr_df_list
d[0]
# (n_epochs, n_cells)
# print(f'{d.shape}')

In [ ]:
LxC_ThetaDeltaPlus
# Look at percent change
(LxC_ThetaDeltaPlus.cell_agg_inst_fr_list - LxC_ThetaDeltaMinus.cell_agg_inst_fr_list) / LxC_ThetaDeltaMinus.cell_agg_inst_fr_list

## NOW: 2023-07-11 - Testing Batch-computed inst_firing_rates

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations


## Load the saved across-session results:
inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

# Figure 3) `PAPER_FIGURE_figure_3`: Firing Rate Index and Long/Short Firing Rate Replays v. Laps

In [ ]:
from neuropy.utils.matplotlib_helpers import FormattedFigureText
from pyphoplacecellanalysis.General.Pipeline.Stages.DisplayFunctions.MultiContextComparingDisplayFunctions.LongShortTrackComparingDisplayFunctions import _plot_long_short_firing_rate_indicies
curr_active_pipeline.reload_default_display_functions()

_out, _out2 = PAPER_FIGURE_figure_3(curr_active_pipeline, defer_render=False, save_figure=True)

In [ ]:
_out2.figures
_out.axes

In [ ]:
1.0-0.090

In [ ]:
fig = _out.figures
fig
# Computed long ($L$)|short($S$) firing rate indicies

# 2023-07-07 - `batch_extended_programmatic_figures` Testing

In [ ]:
curr_active_pipeline.reload_default_display_functions()

neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True)

In [ ]:

batch_extended_programmatic_figures(curr_active_pipeline, write_vector_format=False, write_png=False, debug_print=False)

# 2023-07-19 - Validation with 3D tools

In [ ]:
# curr_active_pipeline.plot.

display_output = {}
# active_config_name = long_epoch_name
active_config_name = global_epoch_name

# 3D Plotters

## 🪟 ipcDataExplorer - 3D Interactive Tuning Curves Plotter

In [ ]:
pActiveTuningCurvesPlotter = None

zScalingFactor = 2000.0 # worked well before with default paramsL
# zScalingFactor = 50.0 # worked well before with default params
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_tuning_curves_plotter', active_config_name, extant_plotter=display_output.get('pActiveTuningCurvesPlotter', None), panel_controls_mode='Qt', should_nan_non_visited_elements=False, zScalingFactor=zScalingFactor, separate_window=False)
ipcDataExplorer = display_output['ipcDataExplorer']
display_output['pActiveTuningCurvesPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveTuningCurvesPlotter = display_output['pActiveTuningCurvesPlotter']
root_dockAreaWindow, placefieldControlsContainerWidget, pf_widgets = display_output['pane'] # for Qt mode:

## 🪟 ipspikesDataExplorer - 3D Interactive Spike and Behavior Plotter

In [ ]:
pActiveSpikesBehaviorPlotter = None
display_output = display_output | curr_active_pipeline.display('_display_3d_interactive_spike_and_behavior_browser', active_config_name, extant_plotter=display_output.get('pActiveSpikesBehaviorPlotter', None)) # Works now!
ipspikesDataExplorer = display_output['ipspikesDataExplorer']
display_output['pActiveSpikesBehaviorPlotter'] = display_output.pop('plotter') # rename the key from the generic "plotter" to "pActiveSpikesBehaviorPlotter" to avoid collisions with others
pActiveSpikesBehaviorPlotter = display_output['pActiveSpikesBehaviorPlotter']

### ✅ Test adding the nearest predicted/decoded position as a red point to the 3D plotter:

In [ ]:
_debug_print = False

def _update_nearest_decoded_most_likely_position_callback(start_t, end_t):
    """ Only uses end_t
    Implicitly captures: ipspikesDataExplorer, _get_nearest_decoded_most_likely_position_callback
    
    Usage:
        _update_nearest_decoded_most_likely_position_callback(0.0, ipspikesDataExplorer.t[0])
        _conn = ipspikesDataExplorer.sigOnUpdateMeshes.connect(_update_nearest_decoded_most_likely_position_callback)

    """
    def _get_nearest_decoded_most_likely_position_callback(t):
        """ A callback that when passed a visualization timestamp (the current time to render) returns the most likely predicted position provided by the active_two_step_decoder
        Implicitly captures:
            active_one_step_decoder, active_two_step_decoder
        Usage:
            _get_nearest_decoded_most_likely_position_callback(9000.1)
        """
        active_time_window_variable = active_one_step_decoder.time_window_centers # get time window centers (n_time_window_centers,) # (4060,)
        active_most_likely_positions = active_one_step_decoder.most_likely_positions.T # (4060, 2) NOTE: the most_likely_positions for the active_one_step_decoder are tranposed compared to the active_two_step_decoder
        # active_most_likely_positions = active_two_step_decoder.most_likely_positions # (2, 4060)
        assert np.shape(active_time_window_variable)[0] == np.shape(active_most_likely_positions)[1], f"timestamps and num positions must be the same but np.shape(active_time_window_variable): {np.shape(active_time_window_variable)} and np.shape(active_most_likely_positions): {np.shape(active_most_likely_positions)}!"
        last_window_index = np.searchsorted(active_time_window_variable, t, side='left') # side='left' ensures that no future values (later than 't') are ever returned
        # TODO: CORRECTNESS: why is it returning an index that corresponds to a time later than the current time?
        # for current time t=9000.0
        #     last_window_index: 1577
        #     last_window_time: 9000.5023
        # EH: close enough
        last_window_time = active_time_window_variable[last_window_index] # If there is no suitable index, return either 0 or N (where N is the length of `a`).
        displayed_time_offset = t - last_window_time # negative value if the window time being displayed is in the future
        if _debug_print:
            print(f'for current time t={t}\n\tlast_window_index: {last_window_index}\n\tlast_window_time: {last_window_time}\n\tdisplayed_time_offset: {displayed_time_offset}')
        return (last_window_time, *list(np.squeeze(active_most_likely_positions[:, last_window_index]).copy()))

    t = end_t # the t under consideration should always be the end_t. This is written this way just for compatibility with the ipspikesDataExplorer.sigOnUpdateMeshes (float, float) signature
    curr_t, curr_x, curr_y = _get_nearest_decoded_most_likely_position_callback(t)
    curr_debug_point = [curr_x, curr_y, ipspikesDataExplorer.z_fixed[-1]]
    if _debug_print:
        print(f'tcurr_debug_point: {curr_debug_point}') # \n\tlast_window_time: {last_window_time}\n\tdisplayed_time_offset: {displayed_time_offset}
    ipspikesDataExplorer.perform_plot_location_point('debug_point_plot', curr_debug_point, color='r', render=True)
    return curr_debug_point

_update_nearest_decoded_most_likely_position_callback(0.0, ipspikesDataExplorer.t[0])
# _conn = pg.SignalProxy(ipspikesDataExplorer.sigOnUpdateMeshes, rateLimit=14, slot=_update_nearest_decoded_most_likely_position_callback)
_conn = ipspikesDataExplorer.sigOnUpdateMeshes.connect(_update_nearest_decoded_most_likely_position_callback)

# 2023-08-29 - Add extended flexitext labels to pf1D Placefields Figure:
`neuropy.plotting.ratemaps.plot_ratemap_1D`

In [ ]:
from neuropy.analyses.placefields import PfND
from neuropy.plotting.ratemaps import plot_ratemap_1D
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum
# Uses using `ax.set_yticklabels(list(sorted_neuron_id_labels))` to label each cell's tuning curve and offsets to plot them.

curr_active_pipeline.reload_default_display_functions()
_out = curr_active_pipeline.display(display_function='_display_1d_placefields', active_session_configuration_context=long_epoch_context)
_out

In [ ]:
curr_active_pipeline.sess.neurons.

In [ ]:
unique_aclu_information_df: pd.DataFrame = curr_active_pipeline.sess.spikes_df.spikes.extract_unique_neuron_identities()
unique_aclu_information_df

In [ ]:
unique_aclu_information_df[np.isin(unique_aclu_information_df['aclu'], [2,8,19,36,37,54,63,68,73,74,97,109])].to_clipboard(index=None)

# unique_aclu_information_df[np.isin(unique_aclu_information_df['aclu'], [4,13,21,23,52,58,67,81])].to_clipboard(index=None)

In [ ]:
from neuropy.analyses.placefields import PfND
from neuropy.plotting.ratemaps import plot_ratemap_1D
from neuropy.core.neuron_identities import PlotStringBrevityModeEnum


# `_display_1d_placefields` kwargs = {'active_config_name': 'maze1'}
active_context = long_epoch_context
active_display_fn_identifying_ctx = active_context.adding_context('display_fn', display_fn_name='1d_placefields') # using short name instead of full name here
# _build_safe_kwargs
# curr_active_pipeline.computation_results[
# pf1D: PfND = computation_result.computed_data['pf1D']
pf1D: PfND = deepcopy(long_pf1D)
# ax_pf_1D = pf1D.plot_ratemaps_1D(**{'active_config_name': 'maze1'})
pf1D.included_neuron_IDs
pf1D.cell_ids

ax, sort_ind, neurons_colors_array = plot_ratemap_1D(pf1D.ratemap, ax=None, pad=1, normalize_tuning_curve=True, sortby=None, cmap=None, brev_mode=PlotStringBrevityModeEnum.MINIMAL, active_context=active_context, use_flexitext_titles=True, use_flexitext_ticks=False, **{'active_config_name': 'maze1'})


In [ ]:
## To file only:
with matplotlib_file_only():
	# Perform non-interactive Matplotlib operations with 'AGG' backend
	# neptuner = batch_perform_all_plots(curr_active_pipeline, enable_neptune=True, neptuner=None)
	main_complete_figure_generations(curr_active_pipeline, enable_default_neptune_plots=True, save_figures_only=True, save_figure=True)
	

In [ ]:
ax.set_position([0.1, 0.1, 0.8, 0.8])  # [left, bottom, width, height]


In [ ]:
ax.reset_position()

In [ ]:
fig = plt.gcf()
fig.tight_layout(rect=[0, 0, 1, 0.85])
fig



In [ ]:
plt.show()

In [ ]:
# Add padding to the top of the figure
fig.subplots_adjust(top=0.85)  # Adjust as needed


In [ ]:
# ax.get_position()

ax.get_clip_box()

In [ ]:
# f"Cell {ratemap.neuron_ids[cell]} - {ratemap.get_extended_neuron_id_string(neuron_i=cell)} \n{round(np.nanmax(pfmap),2)} Hz"
a_label = '87-s10, c10\n1.0 Hz'
neuron_extended_id = pf1D.ratemap.neuron_extended_ids[0]
a_label_lines = a_label.splitlines()
# if len(a_label_lines) == 2:


# a_label_lines

In [ ]:
a_label_subset = '87-s10, c10'
a_label_subset = '87'
try:
	aclu, remainder = a_label_subset.split('-')
	remainder.split(' ') # ['s10,', 'c10']
except ValueError:


In [ ]:
# f'(shank {neuron_extended_id.shank}, cluster {neuron_extended_id.cluster})'
# f's{neuron_extended_id.shank}, c{neuron_extended_id.cluster}'

def _build_flexitext_neuron_extended_id_sublabel(label:str='s', value:str=666):
	return f'<size:8><weight:bold>{label}</></><size:7>{value}</>'

final_label_str = f'<size:12><weight:bold>{neuron_extended_id.id}</></>'
final_label_str = final_label_str + '|'.join([ # _build_flexitext_neuron_extended_id_sublabel('aclu', neuron_extended_id.id),
_build_flexitext_neuron_extended_id_sublabel('s', neuron_extended_id.shank),
_build_flexitext_neuron_extended_id_sublabel('c', neuron_extended_id.cluster),
])

final_label_str

In [ ]:

import re

# Define the Python regex pattern based on the provided regex
pattern = r'(?P<aclu>\d+)-(?:s(?P<shank>\d+))(?:, c(?P<cluster>\d+))?'

# Example usage
text = '87-s10, c10' # "1234-s567, c89"
match = re.search(pattern, text)

if match:
    aclu = match.group('aclu')
    shank = match.group('shank')
    cluster = match.group('cluster')
    print(f"aclu: {aclu}, shank: {shank}, cluster: {cluster}")
else:
    print("No match found.")


In [ ]:
pf1D.neuron_extended_ids

In [ ]:
pf1D.ratemap

In [ ]:
pf1D.ratemap.config

In [ ]:

active_config = deepcopy(pf1D.config)
is_2D = False
subtitle_string = '\n'.join([f'{active_config.str_for_display(is_2D)}'])
subtitle_string

In [ ]:
from flexitext import flexitext ## flexitext version
from neuropy.utils.matplotlib_helpers import FormattedFigureText


In [ ]:

# ytick_locations = list(np.arange(len(sort_ind)) + 0.5)
# ax.set_yticks(ytick_locations) # OLD: ax.set_yticks(list(np.arange(len(sort_ind)) + 0.5))
# ax.set_yticklabels(list(sorted_neuron_id_labels))
# Set the neuron id labels on the y-axis to the color of their cell:
y_tick_label_objects = []
for i, a_tick_label in enumerate(ax.get_yticklabels()):
	# color = neurons_colors_array[:, i]
	# ## Cell color is stroke color mode: black text with stroke colored with cell-specific color:
	# a_tick_label.set_color('black')
	# strokewidth = 0.5
	# a_tick_label.set_path_effects([withStroke(foreground=color, linewidth=strokewidth)])
	# label_text = a_tick_label.get_text()
	# pos_x, pos_y = a_tick_label.get_position()
	# a_flexi_tick_label = flexitext(pos_x, pos_y, f'<size:10><weight:bold>{label_text}</></>\t<size:8>small</>', va="bottom", xycoords="figure fraction")
	# y_tick_label_objects.append(a_flexi_tick_label)
	print(f'{i}: {a_tick_label}')
	a_tick_label.remove()

In [ ]:
y_tick_label_objects

In [ ]:
label_text = a_tick_label.get_text()
pos_x, pos_y = a_tick_label.get_position()

In [ ]:
# a_tick_label.get_position()
a_tick_label.get_color()

In [ ]:
a_tick_label.get_path_effects()

In [ ]:
a_flexi_tick_label = flexitext(text_formatter.left_margin, 0.90, f'<size:22><weight:bold>{title_string}</></>\n<size:10>{subtitle_string}</>', va="bottom", xycoords="figure fraction")


In [ ]:

use_flexitext_titles = kwargs.get('use_flexitext_titles', True) # use fancy flexitext titles if this is true
        
active_pf_occupancy_identifier_string = ' - '.join([active_pf_occupancy_identifier_string] + identifier_details_list)
title_string = ' '.join([active_pf_occupancy_identifier_string])
subtitle_string = ' '.join([f'{self.config.str_for_display(is_2D)}'])    

occupancy_fig, occupancy_ax = plot_placefield_occupancy(self, fig=fig, ax=ax, **kwargs)

occupancy_fig.canvas.manager.set_window_title(title_string) # sets the window's title


if (active_context is None) or (not use_flexitext_titles):
	occupancy_fig.suptitle(title_string, fontsize='14', wrap=True)
	occupancy_ax.set_title(subtitle_string, fontsize='10', wrap=True)
else:
	from flexitext import flexitext ## flexitext version
	# Clear the normal text:
	occupancy_fig.suptitle('')
	occupancy_ax.set_title('')
	text_formatter = FormattedFigureText()
	text_formatter.setup_margins(occupancy_fig)
	active_config = deepcopy(self.config)
	# active_config.float_precision = 1
	
	subtitle_string = '\n'.join([f'{active_config.str_for_display(is_2D)}'])
	header_text_obj = flexitext(text_formatter.left_margin, 0.90, f'<size:22><weight:bold>{title_string}</></>\n<size:10>{subtitle_string}</>', va="bottom", xycoords="figure fraction")
	footer_text_obj = text_formatter.add_flexitext_context_footer(active_context=active_context) # flexitext((text_formatter.left_margin*0.1), (text_formatter.bottom_margin*0.25), text_formatter._build_footer_string(active_context=active_context), va="top", xycoords="figure fraction")



# 2023-03-08 - Build Transition Matrix for each position bin

In [ ]:
from copy import deepcopy
import numpy as np
from neuropy.utils.mixins.binning_helpers import transition_matrix

### 1D Transition Matrix:


def _compute_position_transition_matrix(pf1D, n_powers:int=3):
	"""  1D Transition Matrix """
	num_position_states = len(pf1D.xbin_labels)
	binned_x = pf1D.filtered_pos_df['binned_x'].to_numpy()
	binned_x_indicies = binned_x - 1
	binned_x_transition_matrix = transition_matrix(deepcopy(binned_x_indicies), markov_order=1, max_state_index=num_position_states)
	# binned_x_transition_matrix_higher_order_list = [binned_x_transition_matrix, transition_matrix(deepcopy(binned_x_indicies), markov_order=2, max_state_index=num_position_states), transition_matrix(deepcopy(binned_x_indicies), markov_order=3, max_state_index=num_position_states)]

	binned_x_transition_matrix[np.isnan(binned_x_transition_matrix)] = 0.0
	binned_x_transition_matrix_higher_order_list = [binned_x_transition_matrix] + [np.linalg.matrix_power(binned_x_transition_matrix, n) for n in np.arange(2, n_powers+1)]
	# , np.linalg.matrix_power(binned_x_transition_matrix, 2), np.linalg.matrix_power(binned_x_transition_matrix, 3)
	# binned_x_transition_matrix.shape # (64, 64)
	return binned_x_transition_matrix_higher_order_list

# pf1D = deepcopy(curr_active_pipeline.computation_results['maze1'].computed_data['pf1D'])
# pf1D = deepcopy(global_pf1D)
# pf1D = deepcopy(short_pf1D)
pf1D = deepcopy(long_pf1D)
binned_x_transition_matrix_higher_order_list = _compute_position_transition_matrix(pf1D=pf1D)

In [ ]:
# Visualization ______________________________________________________________________________________________________ #
from pyphoplacecellanalysis.GUI.PyQtPlot.BinnedImageRenderingWindow import BasicBinnedImageRenderingWindow, LayoutScrollability
out = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[0], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix', title="Transition Matrix for binned x (from, to)", variable_label='Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out.add_data(row=1, col=0, matrix=active_eloy_analysis.pf_overlapDensity_2D, xbins=active_pf_2D_dt.xbin_labels, ybins=active_pf_2D_dt.ybin_labels, name='pf_overlapDensity', title='pf overlapDensity metric', variable_label='pf overlapDensity')

# out.add_data(row=1, col=0, matrix=binned_x_transition_matrix_higher_order_list[1], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^2', title='2nd Order Transition Matrix for binned x (from, to)', variable_label='2nd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE
# out.add_data(row=2, col=0, matrix=binned_x_transition_matrix_higher_order_list[2], xbins=pf1D.xbin_labels, ybins=pf1D.ybin_labels, name='binned_x_transition_matrix^3', title='3rd Order Transition Matrix for binned x (from, to)', variable_label='3rd Order Transition Matrix') # , scrollability_mode=LayoutScrollability.NON_SCROLLABLE

# Separate Windows for each:
# out2 = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[1], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix^2', title="2nd Order Transition Matrix for binned x (from, to)", variable_label='2nd Order Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)
# out3 = BasicBinnedImageRenderingWindow(binned_x_transition_matrix_higher_order_list[2], pf1D.xbin_labels, pf1D.xbin_labels, name='binned_x_transition_matrix^3', title="3rd Order Transition Matrix for binned x (from, to)", variable_label='3rd Order Transition Matrix', scrollability_mode=LayoutScrollability.NON_SCROLLABLE)

# 2023-08-30 - Time-binned Unit Firing Rates

In [ ]:
time_binned_instantaneous_unit_specific_spike_rate = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.time_binned_instantaneous_unit_specific_spike_rate
timestamps = time_binned_instantaneous_unit_specific_spike_rate.time_bins

value_df = time_binned_instantaneous_unit_specific_spike_rate.instantaneous_unit_specific_spike_rate_values
value_df

In [ ]:
# Number of spikes version:
time_binned_unit_specific_spike_rate = curr_active_pipeline.global_computation_results.computed_data.jonathan_firing_rate_analysis.time_binned_unit_specific_spike_rate
timestamps = time_binned_unit_specific_spike_rate.time_bins
value_df = time_binned_unit_specific_spike_rate.time_binned_unit_specific_binned_spike_rate
value_df

In [ ]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(custom_InstSpikeRateTrends_df):
    # Sort by column: 'custom_frs_index' (ascending)
    custom_InstSpikeRateTrends_df = custom_InstSpikeRateTrends_df.sort_values(['custom_frs_index'])
    return custom_InstSpikeRateTrends_df

custom_InstSpikeRateTrends_df_clean = clean_data(custom_InstSpikeRateTrends_df.copy())
custom_InstSpikeRateTrends_df_clean.head()